# Een pre-trained netwerk
In de tweede opdracht gebruiken we een vooraf getrained netwerk. Door een pre-trained netwerk te gebruiken is er minder data nodig om een nieuwe classifier te trainen.

In dit geval gebruiken we ResNet 50. Dit netwerk is getrained op de Imagenet dataset (ILSVRC) met 1000 klassen en meer dan een milioen plaatjes.

Bronnen:
* https://www.learnopencv.com/keras-tutorial-transfer-learning-using-pre-trained-models/

In [ ]:
## We beginnen met het laden van de TensorFlow en Keras biblotheken.
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
## Laad een pre-trained netwerk
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions

model = ResNet50(weights='imagenet', input_shape=(224, 224, 3))

In [ ]:
## Download een voorbeeld plaatje
from urllib.request import urlopen
from io import BytesIO
from PIL import Image
from keras.preprocessing.image import img_to_array

def download_image(url):
  img = Image.open(BytesIO(urlopen(url).read()))
  img = img.convert("RGB").resize((224,224))
  return img_to_array(img)

im = download_image("http://farm1.static.flickr.com/8/12567442_838940c1f1.jpg")

In [ ]:
plt.figure()
plt.imshow(im/255)
plt.show()

In [ ]:
# Voorspelling volgens het voorgetrainde netwerk
x = np.expand_dims(np.copy(im), axis=0)
x = preprocess_input(x) # gebruik de preprocessing die voor het trainen ook gebruikt is
preds = model.predict(x)
print('Predicted:', decode_predictions(preds, top=3)[0])

## Vragen / opdrachten
Probeer deze vragen te beantwoorden. Hiervoor moet de bovenstaande code misschien worden aangepast. Eventueel kunnen hieronder extra cellen worden toegevoegd.

1. Download zelf een aantal plaatjes, en kijk wat het netwerk hierop voorspelt.

## Een eigen dataset
Voor de rest van deze opdracht kan je een simpele dataset gebruiken met twee klassen. Die kan je als volgt downloaden en openen:

In [ ]:
!wget https://github.com/twanvl/ou-tensorflow-tutorial/raw/master/office-dataset.tar.gz
!tar -zxf office-dataset.tar.gz

In [ ]:
nTrain = 99*2
ims = np.zeros((nTrain,224,224,3))
y   = np.zeros(nTrain)
j   = 0
for i in range(99):
  for dir in ["mouse","headphones"]:
    img = Image.open("images/{}/frame_{:04d}.jpg".format(dir,i+1))
    img = img.convert("RGB").resize((224,224))
    ims[j,:,:,:] = img_to_array(img)
    y[j] = 0 if dir == "mouse" else 1
    j += 1

In [ ]:
# dit is de voorspelling van het op imagenet getrainde netwerk op deze dataset
x = np.copy(ims)
x = preprocess_input(x)
preds = model.predict(x)
decoded = decode_predictions(preds, top=2)

which = [0,1,2,3,7]
plt.figure(None,(18,18))
for i in range(5):
  plt.subplot(1,5,i+1)
  plt.imshow(ims[which[i]] / 255)
  print('Predicted:', decoded[which[i]])
plt.show()

## Het netwerk aanpassen voor een eigen probleem
In de pratktijk wil je geen honden en katten classificeren, maar heb je vaak een specifiek probleem. Wat je kan doen is een bestaand netwerk nemen, en alleen de laatste laag hiervan aanpassen voor jouw trainingset.

Er zijn twee manieren waarop je een pretrained netwerk kan gebruiken voor nieuwe data:
1. door nieuwe lagen toe te voegen aan het model.
2. door het model te gebruiken als feature extractor, en deze features als invoer te nemen voor een nieuwe classifier.

Hier doen we dat tweede.

We beginnen opnieuw met ResNet 50, maar nu zonder de laatste laag

In [ ]:
feature_extractor = ResNet50(weights='imagenet', input_shape=(224, 224, 3), include_top=False)

Nu gaan we de features van alle testdata extraheren met dit feature extractor netwerk

In [ ]:
x = np.copy(ims)
x = preprocess_input(x)
features = np.squeeze(feature_extractor.predict(x))

In [ ]:
# Splits in train en testset
x_train = features[0:150]
y_train = y[0:150]
x_test = features[150:]
y_test = y[150:]

num_classes = 2
y_train_mat = keras.utils.to_categorical(y_train, num_classes)
y_test_mat = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Train een netwerk, net als voor de MNIST dataset
from keras import models
from keras import layers
from keras import optimizers
 
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=2048))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# Train 10 epochs met batches van 32 samples
history = model.fit(x_train, y_train_mat,
                    batch_size=32,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test_mat))

## Vragen / opdrachten

2. Hoeveel features worden gextraheerd met ResNet50? (hint: kijk naar `features.shape`)
3. Er zitten meerdere voorgetrainde netwerken in Keras. Zie https://keras.io/applications/. Probeer een ander netwerk uit, zijn er verschillen?
3. Vindt of maak een eigen dataset en train hierop een classifier.